In [1]:
import numpy as np
import math
from qiskit import QuantumCircuit, QuantumRegister, execute, Aer, ClassicalRegister
from qiskit.providers.aer import QasmSimulator
import qiskit_create_circ as qcc
from qiskit.providers.ibmq.job import job_monitor

import cirq
import create_circuit as cc
from cirq import GridQubit, X, CNOT, TOFFOLI, ry
from cirq import Simulator
early = cirq.InsertStrategy.EARLIEST
new = cirq.InsertStrategy.NEW_THEN_INLINE

# Parton Shower

link to paper: https://arxiv.org/pdf/1904.03196.pdf

# Indexing

Documentation for Cirq grid qubits - https://quantumai.google/reference/python/cirq/devices/GridQubit

**Explaination of my register indexing**

*Note: If there are any bugs in the code I would recommend checking indexing first*

When comparing the parton shower algorithm in Qiskit v.s. Cirq, we see that the quantum registers in Cirq use Grid qubits. Grid Qubits represent a 2 dimensional square lattice and are 2D arrays with the elements being qubits. For example the $|p \rangle $ register in Cirq for 1 step and 1 particle is in the following form <br>
[[cirq.GridQubit(0, 0), cirq.GridQubit(0, 1), cirq.GridQubit(0, 2)], <br>
[cirq.GridQubit(1, 0), cirq.GridQubit(1, 1), cirq.GridQubit(1, 2)], <br>
[cirq.GridQubit(2, 0), cirq.GridQubit(2, 1), cirq.GridQubit(2, 2)]]

However, Qiskit does not have a grid qubit register equivalent. The registers are their QuantumRegister objects and can be thought of linearly. So we need to create a mapping between the two. <br>
Let $gr$ be the grid qubit register and $lr$ be the linear qubit register

$gr = \begin{bmatrix} q_1 & q_2 & ... & q_m \\ \\ & \ddots & \\ \\ q_{m^2 - m} & q_{m^2 - m + 1} & ... & q_n \end{bmatrix}$ <br> 

and 

$lr = \begin{bmatrix} q_1 \\ q_2 \\ .\\.\\. \\ q_n  \end{bmatrix}$ <br>

Here the length of each sub array is m and the total number of qubits is n. 

Then we need $gr \mapsto lr$
 
Now say the Cirq code is using qubit $gr[i][j] = q_k$. Then to get qubit $q_k$ in the qiskit code we need to do $gr[i*m + j]$, which then gives us the desired qubit in the Qiskit code $q_k$


# Resetting and Remeasuring using Qiskit

Documentation (from IBM):<br>
https://quantum-computing.ibm.com/lab/docs/iql/manage/systems/reset/backend_reset <br>
https://qiskit.org/documentation/stubs/qiskit.circuit.Reset.html

In [2]:
#toy example
backend = Aer.get_backend('qasm_simulator')

qc = QuantumCircuit(3, 6)
qc.h(0)
qc.h(1)
qc.h(2)
qc.cx(0, 2)
qc.reset([1])
qc.x(1)
qc.measure(range(2), range(2))
qc.reset([0])
qc.reset([1])
qc.measure(range(2), range(2,4))



circs0 = [qc]
job = execute(circs0, backend)
result = job.result()
job_monitor(job)

counts = result.get_counts(qc)
print(counts)

qc.draw()

Job Status: job has successfully run
{'000010': 541, '000011': 483}


┌───┐               ┌─┐          ┌─┐   
q_0: ┤ H ├──■────────────┤M├─|0>──────┤M├───
     ├───┤  │       ┌───┐└╥┘ ┌─┐      └╥┘┌─┐
q_1: ┤ H ├──┼───|0>─┤ X ├─╫──┤M├──|0>──╫─┤M├
     ├───┤┌─┴─┐     └───┘ ║  └╥┘       ║ └╥┘
q_2: ┤ H ├┤ X ├───────────╫───╫────────╫──╫─
     └───┘└───┘           ║   ║        ║  ║ 
c: 6/═════════════════════╩═══╩════════╩══╩═
                          0   1        2  3

# Using class - QuantumPartonShower

Note: I created QuantumPartonShower.py, which does the same thing as qiskit_create_circ.py except the former is a class and the latter just a script.

In [3]:
from QuantumPartonShower import QuantumPartonShower as qpc

In [4]:
g_1, g_2, g_12 = 2, 1, 0
eps = .001
reps = 1000

In [5]:
oneN_oneF = qpc(1, 1, 0)

In [6]:
oneN_oneF.createCircuit(eps, g_1, g_2, g_12, [[0, 0, 1]])

k:  0
generated circuit on 11 qubits


(<qiskit.circuit.quantumcircuit.QuantumCircuit at 0x1a27a32850>,
 {'pReg': QuantumRegister(6, 'q5'),
  'hReg': QuantumRegister(1, 'q6'),
  'w_hReg': QuantumRegister(1, 'q7'),
  'eReg': QuantumRegister(1, 'q8'),
  'wReg': QuantumRegister(5, 'q9'),
  'n_aReg': QuantumRegister(2, 'q10'),
  'w_aReg': QuantumRegister(2, 'q11'),
  'n_bReg': QuantumRegister(2, 'q12'),
  'w_bReg': QuantumRegister(2, 'q13'),
  'n_phiReg': QuantumRegister(2, 'q14'),
  'w_phiReg': QuantumRegister(2, 'q15')})

In [7]:
circuit = oneN_oneF._circuit

In [8]:
oneN_oneF.simulate('qasm', shots=1000)

{'00000000000000000000000100': 110, '00000000000000000001001100': 890}

In [9]:
oneN_oneF.simulate('statevector', position=True)

position of non zero element:  76 
value:  (1+2.7757447698399526e-16j) 
absolute value:  1.0


array([0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j])

# Defining initial variables and circuit

Note: everything from this point and on was using the script qiskit_create_circ becuase I wrote it before I created the class version (QuantumPartonShower), but they do the same thing.

In [ ]:
N = 1
m = 0
n_i = 2
L = int(math.floor(math.log(N + n_i, 2)) + 1)
l = int(math.floor(math.log(m + n_i, 2)) + 1)

g_1, g_2, g_12 = 2, 1, 0
gp = math.sqrt(abs((g_1 - g_2) ** 2 + 4 * g_12 ** 2))
if g_1 > g_2:
    gp = -gp
g_a, g_b = (g_1 + g_2 - gp) / 2, (g_1 + g_2 + gp) / 2
u = math.sqrt(abs((gp + g_1 - g_2)/ (2 * gp)))
eps = .001

#qiskit
pReg, hReg, w_hReg, eReg, wReg, n_aReg, w_aReg, n_bReg, w_bReg, n_phiReg, w_phiReg = qcc.allocateQubits(N, n_i, L)
timeStepList, P_aList, P_bList, P_phiList, Delta_aList, Delta_bList, Delta_phiList = [], [], [], [], [], [], []
qcc.populateParameterLists(N, timeStepList, P_aList, P_bList, P_phiList, Delta_aList, Delta_bList, 
                       Delta_phiList, g_a, g_b, eps)
circuit = QuantumCircuit(pReg, hReg, w_hReg, eReg, wReg, n_aReg, w_aReg, n_bReg, w_bReg, n_phiReg, w_phiReg)
# circuit = QuantumCircuit(pReg, hReg, eReg, wReg, n_aReg, n_bReg, n_phiReg)

simulator = Aer.get_backend('statevector_simulator')

#cirq
pRegC, hRegC, w_hRegC, eRegC, wRegC, n_aRegC, w_aRegC, n_bRegC, w_bRegC, n_phiRegC, w_phiRegC = [], [], [], [], [], [], [], [], [], [], []
cc.allocateQubs(N, n_i, L, pRegC, hRegC, w_hRegC, eRegC, wRegC, n_aRegC, w_aRegC, n_bRegC, w_bRegC, n_phiRegC, w_phiRegC)
qubits = {'pRegC': pRegC, 'hRegC': hRegC, 'w_hRegC': w_hRegC, 'eRegC': eRegC, 'wRegC': wRegC, 'n_aRegC': n_aRegC,
               'w_aRegC': w_aRegC, 'n_bRegC': n_bRegC, 'w_bRegC': w_bRegC, 'n_phiRegC': n_phiRegC, 'w_phiRegC': w_phiRegC}
timeStepListC, P_aListC, P_bListC, P_phiListC, Delta_aListC, Delta_bListC, Delta_phiListC = [], [], [], [], [], [], []
cc.populateParameterLists(N, timeStepListC, P_aListC, P_bListC, P_phiListC, Delta_aListC, Delta_bListC,
                       Delta_phiListC, g_a, g_b, eps)
circuitC = cirq.Circuit()
simulatorC = Simulator()


#### uncomment to add x gates on all qubits

In [ ]:
# #qiskit
# [[circuit.h(qubit) for qubit in Reg] for Reg in circuit.qregs]
# [[print(qubit) for qubit in Reg] for Reg in circuit.qregs]

circuit.x(eReg[0])
# circuit.x(n_aReg[0])
circuit.x(n_bReg[0])
circuit.x(n_bReg[1])
# circuit.x(n_phiReg[0])
circuit.x(pReg[1])

# #cirq
# circuitC.append(X(eRegC[0]))
# circuitC.append(X(n_bReg[0]))
# circuitC.append(X(n_bReg[1]))
# circuitC.append(X(pReg[1]))
# circuitC.append([[X((qubit)) for qubit in pRegC[i]] for i in range(N+n_i)])
# circuitC.append([[X((qubit)) for qubit in hRegC[i]] for i in range(N)])
# circuitC.append(X(w_hRegC[0]))
# circuitC.append([X(eRegC[j]) for j in range(len(eRegC))])
# circuitC.append([X(wRegC[j]) for j in range(len(wRegC))])
# circuitC.append([X(n_phiRegC[j]) for j in range(len(n_phiRegC))])
# circuitC.append([X(w_phiRegC[j]) for j in range(len(w_phiRegC))])
# circuitC.append([X(n_aRegC[j]) for j in range(len(n_aRegC))])
# circuitC.append([X(w_aRegC[j]) for j in range(len(w_aRegC))])
# circuitC.append([X(n_bRegC[j]) for j in range(len(n_bRegC))])
# circuitC.append([X(w_bRegC[j]) for j in range(len(w_bRegC))])

### qiskit registers

In [ ]:
print("pReg: ", pReg)
print("hReg: ", hReg)
print("w_hReg: ", w_hReg)
print("eReg: ", eReg)
print("wReg: ", wReg)
print("n_aReg: ", n_aReg)
print("w_aReg: ", w_aReg)
print("n_bReg: ", n_bReg)
print("w_bReg: ", w_bReg)
print("n_phiReg: ", n_phiReg)
print("w_phiReg: ", w_phiReg)

### cirq registers

In [ ]:
print("pReg: ", pRegC)
print("hReg: ", hRegC)
print("w_hReg: ", w_hRegC)
print("eReg: ", eRegC)
print("wReg: ", wRegC)
print("n_aReg: ", n_aRegC)
print("w_aReg: ", w_aRegC)
print("n_bReg: ", n_bRegC)
print("w_bReg: ", w_bRegC)
print("n_phiReg: ", n_phiRegC)
print("w_phiReg: ", w_phiRegC)

# Test uCount

### Qiskit

In [ ]:
qcc.uCount(circuit, m, n_i, l, pReg, wReg, n_aReg, w_aReg, n_bReg, w_bReg, n_phiReg, w_phiReg)
# print(circuit)

In [ ]:
result = execute(circuit, simulator).result()
statevector = result.get_statevector(circuit)
print(statevector)

### Cirq

In [ ]:
cc.uCount(circuitC, m, n_i, l, pRegC, wRegC, n_aRegC, w_aRegC, n_bRegC, w_bRegC, n_phiRegC, w_phiRegC)


In [ ]:
resultC = simulatorC.simulate(circuitC)
statevectorC = resultC.final_state_vector
print(statevectorC)

# Test uE

### Qiskit

In [ ]:
qcc.uE(circuit, l, n_i, m, n_phiReg, w_phiReg, n_aReg, w_aReg, n_bReg, w_bReg, wReg, eReg,
           Delta_phiList[0], Delta_aList[0], Delta_bList[0])

In [ ]:
result = execute(circuit, simulator).result()
statevector = result.get_statevector(circuit)

print(statevector)

### Cirq

In [ ]:
cc.uE(circuitC, l, n_i, m, n_phiRegC, w_phiRegC, n_aRegC, w_aRegC, n_bRegC, w_bRegC, wRegC, eRegC,
           Delta_phiListC[0], Delta_aListC[0], Delta_bListC[0])

In [ ]:
resultC = simulatorC.simulate(circuitC)
statevectorC = resultC.final_state_vector
print(statevectorC)

# Test U_h

### Qiskit

In [ ]:
qcc.U_h(circuit, l, n_i, m, n_phiReg, w_phiReg, n_aReg, w_aReg, n_bReg, w_bReg, wReg, eReg, pReg, hReg, w_hReg,
            P_phiList[0], P_aList[0], P_bList[0])

In [ ]:
result = execute(circuit, simulator).result()


In [ ]:
print(result)

In [ ]:
statevector = result.get_statevector(circuit)

print(statevector)

In [ ]:
circuit.draw()

In [ ]:
print(len(statevector))

In [ ]:
[print("position of non zero element: ", list(statevector).index(i), "\nvalue: ", 
       i, "\nabsolute value: ", abs(i)) for i in statevector if abs(i) > 10**(-5)]


In [ ]:
# cr = ClassicalRegister(2,'cr')
circuit.measure_all()

In [ ]:
circuit.draw()

In [ ]:
print(result)

In [ ]:
job = execute(circuit, simulator, shots=100)
result = job.result()
counts = result.get_counts(circuit)

In [ ]:
print(counts)

### Cirq

In [ ]:
cc.U_h(circuitC, l, n_i, m, n_phiRegC, w_phiRegC, n_aRegC, w_aRegC, n_bRegC, w_bRegC, wRegC, eRegC, pRegC, hRegC, w_hRegC,
            P_phiListC[0], P_aListC[0], P_bListC[0])


In [ ]:
resultC = simulatorC.simulate(circuitC)
statevectorC = resultC.final_state_vector
print(statevectorC)

In [ ]:
print(len(statevectorC))

# Test U_p

### Qiskit

In [ ]:
qcc.U_p(circuit, l, n_i, m, pReg, hReg, w_hReg, wReg, g_a, g_b)

In [ ]:
result = execute(circuit, simulator).result()
statevector = result.get_statevector(circuit)

print(statevector)

### Cirq

In [ ]:
cc.U_p(circuitC, l, n_i, m, pRegC, hRegC, w_hRegC, wRegC, g_a, g_b)

In [ ]:
resultC = simulatorC.simulate(circuitC)
statevectorC = resultC.final_state_vector
print(statevectorC)

# Consolodated Testing (aka CreatCircuit())

## n_i fermion(s), N step(s)

In [ ]:
n_i, N = 2, 1
g_1, g_2, g_12 = 2, 1, 0
eps = .001
reps = 1000

### Qiskit

In [ ]:
circ, qubits = qcc.createCircuit(n_i, N, eps, g_1, g_2, g_12, [[0, 0, 1],[0, 0, 1]])

#### statevector simulator

In [ ]:
simulator_sv = Aer.get_backend('statevector_simulator')

In [ ]:
print(qubits)

In [ ]:
result = execute(circ, simulator_sv).result()
statevector = result.get_statevector(circ)

print(statevector)

In [ ]:
# print(abs(complex(-2, 3)))
[print("position of non zero element: ", list(statevector).index(i), "\nvalue: ", 
       i, "\nabsolute value: ", abs(i)) for i in statevector if abs(i) > 10**(-5)]

#### qasm simulator

In [ ]:
simulator_qasm = Aer.get_backend('qasm_simulator')

In [ ]:
circ.measure_all()
#measure out individual registers

In [ ]:
job = execute(circ, simulator_qasm, shots=8000)
result = job.result()
counts = result.get_counts(circ)

In [ ]:
print(counts)

### Cirq

In [ ]:
circC, qubitsC = cc.createCircuit(n_i, N, eps, g_1, g_2, g_12, [[0, 0, 1],[0, 0, 1]])

#### statevector simulator equv

In [ ]:
resultC = simulatorC.simulate(circC)
statevectorC = resultC.final_state_vector
print(statevectorC)

In [ ]:
print(len(statevectorC))

In [ ]:
[print("position of non zero element: ", list(statevectorC).index(i), "\nvalue: ", 
       i, "\nabsolute value: ", abs(i)) for i in statevectorC if abs(i) > 10**(-5)]


In [ ]:
print(len(statevectorC))

#### qasm simulator equiv

In [ ]:
def reverse(lst):
    """reverse a list in place"""
    lst.reverse()
    return lst

In [ ]:
circC.append(cirq.measure(*reverse(qubitsC['pReg'][0]), key='p0'))
circC.append(cirq.measure(*reverse(qubitsC['pReg'][1]), key='p1'))
circC.append(cirq.measure(*reverse(qubitsC['hReg'][0]), key='h0'))
circC.append(cirq.measure(*reverse(qubitsC['w_hReg']), key='w_h'))
circC.append(cirq.measure(*reverse(qubitsC['eReg']), key='e'))
circC.append(cirq.measure(*reverse(qubitsC['wReg']), key='w'))
circC.append(cirq.measure(*reverse(qubitsC['n_aReg']), key='n_a'))
circC.append(cirq.measure(*reverse(qubitsC['n_bReg']), key='n_b'))
circC.append(cirq.measure(*reverse(qubitsC['n_phiReg']), key='n_phi'))
circC.append(cirq.measure(*reverse(qubitsC['w_aReg']), key='w_a'))
circC.append(cirq.measure(*reverse(qubitsC['w_bReg']), key='w_b'))
circC.append(cirq.measure(*reverse(qubitsC['w_phiReg']), key='w_phi'))



In [ ]:
simulator_c = cirq.Simulator()
result = simulator_c.run(circC, repetitions=8000)

In [ ]:
# [result.measurements['p0'], result.measurements['p1']]
# result.measurements['p1'] == [0,0,0]
count = 0
for i in result.measurements['p1']:
#     print(i[2])
    if i[0] == 1 and i[1]== 0 and i[2]== 0:
        count += 1
print(count)


In [ ]:
print(result.measurements)